# Import 

In [1]:
# STANDARD PACKAGES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from datetime import datetime as dt
import time
import json
import random
from tqdm import tqdm #https://pypi.org/project/tqdm/#ipython-jupyter-integ½ration
from functools import reduce
import pickle
import itertools
import warnings
import platform
import multiprocessing as mp


# SCRAPE PACKAGES
import requests
from bs4 import BeautifulSoup
# from pytrends.request import TrendReq #pip install pytrends

# MODEL PACKAGES
    #SKLEARN
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from statsmodels.tsa.x13 import x13_arima_analysis as x13
from sklearn.metrics import max_error, r2_score


# CUSTOM FUNCTIONS

import os
import sys
currentdir = os.path.dirname(os.path.realpath('analysis_DK'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from func import (chunks, reindex, global_id, term_list, time_corr_plot, rmse, time_variable_plot, find_highest_corr, test_train_split,
                  bootstrap_all_windows, bootstrap_n_samples, bootstrap_sample, final_model, final_model_boot, ar_1, grid_bestpar, tuning_window, tuning_window_mp, tuning_window_bestpar,
                  model_tuning, seasadj, seasadj_col_list, abs_percentage_change, add_poly_terms, create_interaction)

/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


## Baseline -  AR(1) 

### Preprocessing

In [264]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])

#### Create extra lag

In [265]:
df_analysis['target_12_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(12)

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [266]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID'], prefix_sep='_', columns=['ID']).copy()

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [267]:
df_analysis = abs_percentage_change(df_analysis,
                                    var_abs_change =  ['target_actual', 'target_lag', 'target_12_lag'],
                                    var_pct_change =  [])

#### Adding interaction terms by regions and all variables

In [268]:
# relevant interaction variables
interaction_1 = ['target_lag', 'target_12_lag'] 

interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [269]:
for var1 in interaction_1:
    for var2 in interaction_2:
        name = var1 + "*" + var2
        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop na

In [270]:
df_analysis.dropna(inplace=True)

In [271]:
df_analysis.sort_index(axis=1, inplace=True)

In [272]:
window = 35
testsize = 1
valsize = 1
rolling_window = True
params = []
n_components = []

#### Select relevant columns

In [273]:
df_ar = df_analysis[['date', 'target_actual', 
                     'target_lag*ID_Capital','target_lag*ID_Central Denmark','target_lag*ID_North Denmark', 
                     'target_lag*ID_Southern Denmark','target_lag*ID_Zealand']]

### Running baseline model

In [274]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_ar, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= True)

#### Concatting val and train

In [275]:
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

#### Estimating with OLS

In [276]:
results_ols= tuning_window(X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, params = params, n_components = n_components, model_str = 'ols')

Tuning params for window: 100%|█████████████████████████████████████████████████████| 103/103 [00:00<00:00, 434.61it/s]


In [277]:
results_ols[1]['best'][1]

0.22869193252058517

#### Exporting the results

In [278]:
with open('results/final/baseline/results_baseline_ar1.pickle', 'wb') as handle:
    pickle.dump(results_ols, handle, protocol= pickle.HIGHEST_PROTOCOL)

### Check results

In [279]:
temp = []
for key in results_ols.keys():
    temp.append(results_ols[key]['best'][1])
    

In [280]:
np.mean(temp)

0.21441759587971718

In [281]:
results_ols[1]['y_pred_dict'][results_ols[1]['best'][0]]

array([0.07, 0.06, 0.  , 0.01, 0.07])

## Baseline -  AR(1) augmented with GT PCA1 + Jobs

### Preprocessing

In [ ]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])
df_PCA = pd.read_csv('OLD/df_PCA.csv', sep = ';', parse_dates=['date'], index_col=0)

In [ ]:
#Adding GT PC1
df_analysis = pd.merge(df_analysis, df_PCA[['ID', 'date', '0']], how = 'left', on =['ID', 'date']).copy()

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [ ]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID'], prefix_sep='_', columns=['ID']).copy()

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [ ]:
df_analysis = abs_percentage_change(df_analysis,
                                    var_abs_change =  ['target_actual', 'target_lag'],
                                    var_pct_change =  [])

#### Adding interaction terms by regions and all variables

In [ ]:
# relevant interaction variables
interaction_1 = ['target_lag', '0'] 

interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [ ]:
for var1 in interaction_1:
    for var2 in interaction_2:
        name = var1 + "*" + var2
        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop na

In [ ]:
df_analysis.dropna(inplace=True)

In [ ]:
df_analysis.sort_index(axis=1, inplace=True)

In [ ]:
window = 35
testsize = 1
valsize = 1
rolling_window = True
params = []
n_components = []

#### Select relevant columns

In [ ]:
df_ar = df_analysis[['date', 'target_actual', 
                     'target_lag*ID_Capital','target_lag*ID_Central Denmark','target_lag*ID_North Denmark', 'target_lag*ID_Southern Denmark','target_lag*ID_Zealand',
                    '0', 'jobs']].copy() #'0*ID_Capital', '0*ID_Central Denmark', '0*ID_North Denmark', '0*ID_Southern Denmark', '0*ID_Zealand'

### Running baseline model

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_ar, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= True)

#### Concatting val and train

In [ ]:
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

#### Estimating with OLS

In [ ]:
results_ols= tuning_window(X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, params = params, n_components = n_components, model_str = 'ols')

#### Exporting the results

In [ ]:
with open('results/model 1/ar1/results_ar1_augmented.pickle', 'wb') as handle:
    pickle.dump(results_ols, handle, protocol= pickle.HIGHEST_PROTOCOL)

### Check results

In [ ]:
temp = []
for key in results_ols.keys():
    temp.append(results_ols[key]['best'][1])
    

In [ ]:
np.mean(temp)

## Baseline -  AR(2) with y_t-1 + y_t-2

### Preprocessing

In [65]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])

#### Create extra lag

In [66]:
df_analysis['target_2_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(2)

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [67]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID'], prefix_sep='_', columns=['ID']).copy()

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [68]:
df_analysis = abs_percentage_change(df_analysis,
                                    var_abs_change =  ['target_actual', 'target_lag', 'target_2_lag'],
                                    var_pct_change =  [])

#### Adding interaction terms by regions and all variables

In [69]:
# relevant interaction variables
interaction_1 = ['target_lag', 'target_2_lag'] 

interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [70]:
for var1 in interaction_1:
    for var2 in interaction_2:
        name = var1 + "*" + var2
        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop na

In [71]:
df_analysis.dropna(inplace=True)

In [72]:
df_analysis.sort_index(axis=1, inplace=True)

In [73]:
window = 35
testsize = 1
valsize = 1
rolling_window = True
params = []
n_components = []

#### Select relevant columns

In [74]:
df_ar = df_analysis[['date', 'target_actual', 
                     'target_lag*ID_Capital','target_lag*ID_Central Denmark','target_lag*ID_North Denmark', 
                     'target_lag*ID_Southern Denmark','target_lag*ID_Zealand',
                     'target_2_lag*ID_Capital','target_2_lag*ID_Central Denmark','target_2_lag*ID_North Denmark', 
                     'target_2_lag*ID_Southern Denmark','target_2_lag*ID_Zealand']]

### Running baseline model

In [75]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_ar, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= True)

#### Concatting val and train

In [76]:
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

#### Estimating with OLS

In [77]:
results_ols= tuning_window(X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, params = params, n_components = n_components, model_str = 'ols')

Tuning params for window: 100%|██████████| 113/113 [00:00<00:00, 392.50it/s]


In [78]:
results_ols[1]['best'][1]

0.402367990774614

#### Exporting the results

In [79]:
with open('results/final/ar1/results_ar2_five.pickle', 'wb') as handle:
    pickle.dump(results_ols, handle, protocol= pickle.HIGHEST_PROTOCOL)

### Check results

In [80]:
temp = []
for key in results_ols.keys():
    temp.append(results_ols[key]['best'][1])
    

In [81]:
np.mean(temp)

0.228357789924281

## Baseline -  AR(2) with y_t-1 + y_t-12

### Preprocessing

In [254]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])

#### Create extra lag

In [255]:
df_analysis['target_12_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(12)

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [256]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID'], prefix_sep='_', columns=['ID']).copy()

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [257]:
df_analysis = abs_percentage_change(df_analysis,
                                    var_abs_change =  ['target_actual', 'target_lag', 'target_12_lag'],
                                    var_pct_change =  [])

#### Adding interaction terms by regions and all variables

In [258]:
# relevant interaction variables
interaction_1 = ['target_lag', 'target_12_lag'] 

interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [259]:
for var1 in interaction_1:
    for var2 in interaction_2:
        name = var1 + "*" + var2
        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop na

In [260]:
df_analysis.dropna(inplace=True)

In [261]:
df_analysis.sort_index(axis=1, inplace=True)

In [262]:
window = 35
testsize = 1
valsize = 1
rolling_window = True
params = []
n_components = []

#### Select relevant columns

In [263]:
df_ar = df_analysis[['date', 'target_actual', 
                     'target_lag*ID_Capital','target_lag*ID_Central Denmark','target_lag*ID_North Denmark', 
                     'target_lag*ID_Southern Denmark','target_lag*ID_Zealand',
                     'target_12_lag*ID_Capital','target_12_lag*ID_Central Denmark','target_12_lag*ID_North Denmark', 
                     'target_12_lag*ID_Southern Denmark','target_12_lag*ID_Zealand']]

### Running baseline model

In [64]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_ar, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= True)

#### Concatting val and train

In [65]:
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

#### Estimating with OLS

In [66]:
results_ols= tuning_window(X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, params = params, n_components = n_components, model_str = 'ols')


Tuning params for window: 100%|█████████████████████████████████████████████████████| 103/103 [00:00<00:00, 624.24it/s]


In [67]:
results_ols[1]['best'][1]

0.14498275759551524

#### Exporting the results

In [61]:
with open('results/final/baseli/results_ar2_12_five.pickle', 'wb') as handle:
    pickle.dump(results_ols, handle, protocol= pickle.HIGHEST_PROTOCOL)

### Check results

In [68]:
temp = []
for key in results_ols.keys():
    temp.append(results_ols[key]['best'][1])
    

In [69]:
np.mean(temp)

0.10561074656657451

# Final models

## ML - Data and preprocessing 

### Import data frame with adjusted here

In [42]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])

### Initial preprocessing and feature construction

- Create dummies 
- Create interaction terms

Overall monthly time trend variable, $t=1,2...,T$ within `ID` variable:

In [43]:
#Temp container
temp = {}

for i in df_analysis['ID'].unique():
    temp[i] = df_analysis[df_analysis['ID']==i]
    temp[i]['t'] = range(1, len(temp[i]['ID'])+1)

#Concatting the df's
temp = pd.concat(temp, ignore_index=True, sort = False)

#Merging onto analysis
df_analysis = pd.merge(left = df_analysis, right = temp[['date', 'ID', 't']], left_on =['date', 'ID'], right_on = ['date', 'ID'])

C:\Users\WSCHUPPLI\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Drop sector variables

In [44]:
df_analysis.drop(['sector_sales_communication', 'sector_other'], axis = 1, inplace=True)

#### Dropping socioeconomic index

In [45]:
df_analysis.drop(['w_ave_socio_index', 'target_lag'], axis = 1, inplace = True)

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [46]:
df_analysis['target_actual'] = df_analysis.groupby(['ID'])['target_actual'].diff()

### Lagged variables

In [47]:
df_analysis['target_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(1)

In [48]:
df_analysis['target_12_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(12)

#### Create new variables with 3 month lag of jobrate

In [49]:
columns_3m_lag = ['jobs', 'sector_information_technology', 'sector_engineering_technology',
                   'sector_management_staff', 'sector_trade_service', 'sector_industry_craft',
                   'sector_teaching', 'sector_office_finance', 'sector_social_health']

for colname in columns_3m_lag:
    df_analysis[str(colname + '_3_lag')] = df_analysis.groupby(['ID'])[colname].shift(3)

#### Dropping some GT variables

In [50]:
# Dropping some GT's
drop_list = ['GT_DK_1', 'GT_DK_4', 'GT_DK_5', 'GT_DK_6', 'GT_DK_8', 'GT_DK_11', 'GT_DK_13', 'GT_DK_15', 'GT_DK_16', 'GT_DK_18', 'GT_DK_19', 'GT_DK_21']

In [51]:
df_analysis.drop(drop_list, axis = 1, inplace=True)

#### Create new variables with 1 month lag of GT

In [52]:
columns_1m_lag = ['GT_DK_0', 'GT_DK_2', 'GT_DK_3', 'GT_DK_7', 'GT_DK_9', 'GT_DK_10', 'GT_DK_12', 'GT_DK_14', 'GT_DK_17', 'GT_DK_20']
for colname in columns_1m_lag:
    df_analysis[str(colname + '_1_lag')] = df_analysis.groupby(['ID'])[colname].shift(1)

#### Month dummies for season effects

In [53]:
df_analysis['month'] = pd.DatetimeIndex(df_analysis['date']).month.astype(str)

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [54]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID','M'], prefix_sep='_', columns=['ID', 'month']).copy()

#### Drop na

In [55]:
df_analysis.dropna(inplace=True)

In [56]:
df_analysis.date.min()

Timestamp('2008-03-01 00:00:00')

#### Adding interaction terms

Polynominal features - To be deleted later

In [57]:
#df_analysis = add_poly_terms(df = df_analysis, 
#                            poly_columns = ['target_actual', 'GT_0', 'GT_1', 'GT_2', 'GT_3', 'GT_4', 'GT_5', 'GT_6', 'GT_7', 'GT_8', 'GT_9', 'GT_10', 'GT_11', 'GT_12', 'GT_13', 'GT_14', 'GT_15', 'GT_16', 'GT_17', 'GT_18', 'GT_19', 'target_lag', 'jobs', 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'])

In [58]:
#df_analysis.dropna(inplace=True)

Adding interaction terms by regions and all variables

In [59]:
# relevant interaction variables
interaction_1 = ['target_lag', 'target_12_lag'] 
# 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'

# get list of all ID area 
interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [60]:
for var1 in interaction_1:
    for var2 in interaction_2:
        name = var1 + "*" + var2
        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop variables to not end up in dummytrap

In [61]:
df_analysis = df_analysis.drop(['ID_Capital', 'M_1'], axis = 1)

In [62]:
df_analysis.drop(interaction_1, axis = 1, inplace=True)

In [63]:
df_analysis.sort_index(axis=1, inplace=True)

#### Setting window size

In [64]:
window = 35
testsize = 1
valsize = 1
rolling_window = True

## Lasso

### Test/train data split

Data must be split non-randomly as it needs to adhere to the underlying time structure. Two distinct approaches:

1. Rolling window (fixed length)
1. Expanding window (initial length that increases with each iteration)

Each model must be run as a loop over the test/train splits. Thus, we will have multiple test/train splits for both rolling window and expanding window.

In [25]:
X_train, X_val, X_test, y_train, y_val, y_test, y_dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [ ]:
with open('results/final/y_dates.pickle', 'wb') as handle:
    pickle.dump(y_dates, handle, protocol= pickle.HIGHEST_PROTOCOL)

### Pre-processing

- Standardizing

Standardizing features for each window

In [26]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_val[win] = sc.transform(X_val[win])
    X_test[win] = sc.transform(X_test[win])

### Training the models

#### Hyperparameter space - random

In [27]:
alphas = np.logspace(-8, 8, num = 10000) #random.sample(list(np.logspace(-10,5, num = 15000)), k = 15000)
n_components = list(np.arange(0.6, 0.95, 0.05).round(2))#[0.60, 0.70, 0.80, 0.90]
params = [(alpha) for alpha in alphas]
print('Number of param sets: '+ str(len(params)))

Number of param sets: 10000


#### Inner loop - training hyperparameter on validation

##### On random space

In [28]:
results_lasso_mp = tuning_window_mp(X_fit = X_train, y_fit = y_train, X_test = X_val, y_test = y_val, params = params, n_components = n_components, model_str = 'lasso')

100%|██████████████████████████████████████████████████████████████████████████████| 103/103 [1:32:53<00:00, 54.12s/it]


In [29]:
with open('results/final/lasso/results_mp.pickle', 'wb') as handle:
    pickle.dump(results_lasso_mp, handle, protocol= pickle.HIGHEST_PROTOCOL)

#### Outer loop - fitting on train / test split

Importing stored results:

In [43]:
with open('results/final/lasso/results_mp.pickle', 'rb') as handle:
    results_lasso_opt = pickle.load(handle)

##### On full sample

Reloading data and concatting:

In [31]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [32]:
#Concatting val and train
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

Standardizing features for each window

In [33]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_test[win] = sc.transform(X_test[win])

In [34]:
results_final = final_model(inner_results=results_lasso_opt, X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, model_str = 'lasso')

100%|███████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 253.10it/s]


#### Exporting final results

In [35]:
with open('results/final/lasso/results_final.pickle', 'wb') as handle:
    pickle.dump(results_final, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [46]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

0.13734338307525912

## Ridge

In [25]:
X_train, X_val, X_test, y_train, y_val, y_test, y_dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

### Pre-processing

- Standardizing

Standardizing features for each window

In [26]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_val[win] = sc.transform(X_val[win])
    X_test[win] = sc.transform(X_test[win])

### Training the models

#### Hyperparameter space - random

In [27]:
alphas = np.logspace(-8, 8, num = 10000) #random.sample(list(np.logspace(-10,5, num = 15000)), k = 15000)
n_components = list(np.arange(0.6, 0.95, 0.05).round(2))#[0.60, 0.70, 0.80, 0.90]
params = [(alpha) for alpha in alphas]
print('Number of param sets: '+ str(len(params)))

Number of param sets: 10000


#### Inner loop - training hyperparameter on validation

##### On random space

In [28]:
results_mp = tuning_window_mp(X_fit = X_train, y_fit = y_train, X_test = X_val, y_test = y_val, params = params, n_components = n_components, model_str = 'ridge')

100%|██████████████████████████████████████████████████████████████████████████████| 103/103 [2:04:21<00:00, 72.45s/it]


In [29]:
with open('results/final/ridge/results_mp.pickle', 'wb') as handle:
    pickle.dump(results_mp, handle, protocol= pickle.HIGHEST_PROTOCOL)

#### Outer loop - fitting on train / test split

Importing stored results:

In [30]:
with open('results/final/ridge/results_mp.pickle', 'rb') as handle:
    results_opt = pickle.load(handle)

##### On full sample

Reloading data and concatting:

In [31]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [32]:
#Concatting val and train
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

Standardizing features for each window

In [33]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_test[win] = sc.transform(X_test[win])

In [34]:
results_final = final_model(inner_results=results_opt, X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, model_str = 'ridge')

100%|███████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 283.75it/s]


#### Exporting final results

In [35]:
with open('results/final/ridge/results_final.pickle', 'wb') as handle:
    pickle.dump(results_final, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [36]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

0.13259001697715023

## Elastic net

In [25]:
X_train, X_val, X_test, y_train, y_val, y_test, y_dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

### Pre-processing

- Standardizing

Standardizing features for each window

In [26]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_val[win] = sc.transform(X_val[win])
    X_test[win] = sc.transform(X_test[win])

### Training the models

#### Hyperparameter space - random

In [27]:
alphas = np.logspace(-8,8, num = 10000) #2000
n_components= [0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90]
l1_ratio = list(np.arange(0.01,0.99,0.05)) ## 0.0 cannot be included due to a bug in the code
params = [(alpha, ratio) for alpha in alphas for ratio in l1_ratio]
print('Number of param sets: '+ str(len(params)))


Number of param sets: 200000


#### Inner loop - training hyperparameter on validation

##### On random space

In [28]:
results_mp = tuning_window_mp(X_fit = X_train, y_fit = y_train, X_test = X_val, y_test = y_val, params = params, n_components = n_components, model_str = 'elasticnet')

100%|████████████████████████████████████████████████████████████████████████████| 103/103 [28:06:25<00:00, 982.39s/it]


In [29]:
with open('results/final/elastic/results_mp.pickle', 'wb') as handle:
    pickle.dump(results_mp, handle, protocol= pickle.HIGHEST_PROTOCOL)

#### Outer loop - fitting on train / test split

Importing stored results:

In [30]:
with open('results/final/elastic/results_mp.pickle', 'rb') as handle:
    results_opt = pickle.load(handle)

##### On full sample

Reloading data and concatting:

In [31]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [32]:
#Concatting val and train
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

Standardizing features for each window

In [33]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_test[win] = sc.transform(X_test[win])

In [34]:
results_final = final_model(inner_results=results_opt, X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, model_str = 'elasticnet')

100%|███████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 326.59it/s]


#### Exporting final results

In [35]:
with open('results/final/elastic/results_final.pickle', 'wb') as handle:
    pickle.dump(results_final, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [36]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

0.1357748422538827

## ML - Data and preprocessing - new

### Import data frame with adjusted here

In [2]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])

### Initial preprocessing and feature construction

- Create dummies 
- Create interaction terms

Overall monthly time trend variable, $t=1,2...,T$ within `ID` variable:

In [3]:
#Temp container
temp = {}

for i in df_analysis['ID'].unique():
    temp[i] = df_analysis[df_analysis['ID']==i]
    temp[i]['t'] = range(1, len(temp[i]['ID'])+1)

#Concatting the df's
temp = pd.concat(temp, ignore_index=True, sort = False)

#Merging onto analysis
df_analysis = pd.merge(left = df_analysis, right = temp[['date', 'ID', 't']], left_on =['date', 'ID'], right_on = ['date', 'ID'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Drop sector variables

In [4]:
df_analysis.drop(['sector_sales_communication', 'sector_other'], axis = 1, inplace=True)

#### Dropping socioeconomic index

In [5]:
df_analysis.drop(['w_ave_socio_index', 'target_lag'], axis = 1, inplace = True)

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [6]:
df_analysis['target_actual'] = df_analysis.groupby(['ID'])['target_actual'].diff()

### Lagged variables

In [7]:
df_analysis['target_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(1)

In [8]:
df_analysis['target_12_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(12)

#### Create new variables with 3 month lag of jobrate

In [9]:
columns_3m_lag = ['jobs', 'sector_information_technology', 'sector_engineering_technology',
                   'sector_management_staff', 'sector_trade_service', 'sector_industry_craft',
                   'sector_teaching', 'sector_office_finance', 'sector_social_health']

for colname in columns_3m_lag:
    df_analysis[str(colname + '_3_lag')] = df_analysis.groupby(['ID'])[colname].shift(3)

#### Dropping some GT variables

In [10]:
# Dropping some GT's
drop_list = ['GT_DK_1', 'GT_DK_4', 'GT_DK_5', 'GT_DK_6', 'GT_DK_8', 'GT_DK_11', 'GT_DK_13', 'GT_DK_15', 'GT_DK_16', 'GT_DK_18', 'GT_DK_19', 'GT_DK_21']

In [11]:
df_analysis.drop(drop_list, axis = 1, inplace=True)

#### Create new variables with 1 month lag of GT

In [12]:
columns_1m_lag = ['GT_DK_0', 'GT_DK_2', 'GT_DK_3', 'GT_DK_7', 'GT_DK_9', 'GT_DK_10', 'GT_DK_12', 'GT_DK_14', 'GT_DK_17', 'GT_DK_20']
for colname in columns_1m_lag:
    df_analysis[str(colname + '_1_lag')] = df_analysis.groupby(['ID'])[colname].shift(1)

#### Month dummies for season effects

In [13]:
df_analysis['month'] = pd.DatetimeIndex(df_analysis['date']).month.astype(str)

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [14]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID','M'], prefix_sep='_', columns=['ID', 'month']).copy()

#### Drop na

In [15]:
df_analysis.dropna(inplace=True)

In [16]:
df_analysis.date.min()

Timestamp('2008-03-01 00:00:00')

#### Adding interaction terms

Polynominal features - To be deleted later

In [17]:
#df_analysis = add_poly_terms(df = df_analysis, 
#                            poly_columns = ['target_actual', 'GT_0', 'GT_1', 'GT_2', 'GT_3', 'GT_4', 'GT_5', 'GT_6', 'GT_7', 'GT_8', 'GT_9', 'GT_10', 'GT_11', 'GT_12', 'GT_13', 'GT_14', 'GT_15', 'GT_16', 'GT_17', 'GT_18', 'GT_19', 'target_lag', 'jobs', 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'])

In [18]:
#df_analysis.dropna(inplace=True)

Adding interaction terms by regions and all variables

In [19]:
# relevant interaction variables
#interaction_1 = ['target_lag', 'target_12_lag'] 
# 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'

# get list of all ID area 
#interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [20]:
#for var1 in interaction_1:
#    for var2 in interaction_2:
#        name = var1 + "*" + var2
#        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop variables to not end up in dummytrap

In [21]:
df_analysis = df_analysis.drop(['ID_Capital', 'M_1'], axis = 1)

In [22]:
#df_analysis.drop(interaction_1, axis = 1, inplace=True)

In [23]:
df_analysis.sort_index(axis=1, inplace=True)

#### Setting window size

In [24]:
window = 35
testsize = 1
valsize = 1
rolling_window = True

## Random forest

In [25]:
X_train, X_val, X_test, y_train, y_val, y_test, y_dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

### Pre-processing

- Standardizing

Standardizing features for each window

In [26]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_val[win] = sc.transform(X_val[win])
    X_test[win] = sc.transform(X_test[win])

### Training the models

#### Hyperparameter space - random

In [27]:
# Website https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# parameter specification
n_components = [0.9]

#range(1, X_train[1].shape[1] +1)
# Number of trees in random forest
n_estimators = [*range(50, 500, 10)]
# Number of features to consider at every split
max_features = ['auto', "sqrt"] # Consider whether this should be set to 'auto as PCA should do its job'
# Maximum number of levels in tree
max_depth = [*range(3, 11, 1), *range(20, 100, 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 3, 5]
# Method of selecting samples for training each tree
# https://gdcoder.com/random-forest-regression-model-explained-in-depth-part-2-python-code-snippet-using-sklearn/

# Create the random grid
d = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_split': min_samples_split,
     'min_samples_leaf':min_samples_leaf
    }

params = list(d.values())

params = list(itertools.product(*params))

print(len(params))


3510


#### Inner loop - training hyperparameter on validation

##### On random space

In [ ]:
results_mp = tuning_window(X_fit = X_train, y_fit = y_train, X_test = X_val, y_test = y_val, params = params, n_components = n_components, model_str = 'randomforest')

Tuning params for window:  47%|████▋     | 48/103 [26:43:51<33:32:58, 2195.98s/it]

In [ ]:
with open('results/final/randomforest/results_mp_no_int.pickle', 'wb') as handle:
    pickle.dump(results_mp, handle, protocol= pickle.HIGHEST_PROTOCOL)

#### Outer loop - fitting on train / test split

Importing stored results:

In [ ]:
with open('results/final/randomforest/results_mp_no_int.pickle', 'rb') as handle:
    results_opt = pickle.load(handle)

##### On full sample

Reloading data and concatting:

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [ ]:
#Concatting val and train
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

Standardizing features for each window

In [ ]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_test[win] = sc.transform(X_test[win])

In [ ]:
results_final = final_model(inner_results=results_opt, X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, model_str = 'randomforest')

#### Exporting final results

In [ ]:
with open('results/final/randomforest/results_final_no_int.pickle', 'wb') as handle:
    pickle.dump(results_final, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [ ]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

## XGboost

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test, y_dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

### Pre-processing

- Standardizing

Standardizing features for each window

In [66]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_val[win] = sc.transform(X_val[win])
    X_test[win] = sc.transform(X_test[win])

### Training the models

#### Hyperparameter space - random

In [69]:
# Website https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# parameter specification
n_components = [0.9]


colsample_bytree = [0.3, 0.5, 0.7, 0.9, 1]#is the subsample ratio of columns when constructing each tree. Subsampling will occur once in every boosting iteration. This number ranges from 0 to 1.
#learning_rate is the step size shrinkage and is used to prevent overfitting. This number ranges from 0 to 1.
# Maximum number of levels in tree
# First try: colsample_bytree = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

max_depth = [*range(3, 11, 1), *range(20, 100, 20)]

#max_depth = [*range(10, 300, 10)]
# first try max_depth = [*range(50, 500, 20)]

n_estimators = [*range(50, 500, 10)]#is the number of boosted trees to fit
# first try n_estimators = [*range(50, 500, 20)]

gamma = [0]

subsample = [0.5, 0.75, 1]

min_child_weight = [1, 3, 5]

# Create the random grid
d = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'colsample_bytree': colsample_bytree,
    'gamma' : gamma,
    'subsample' : subsample,
    'min_child_weight' : min_child_weight
    }

params = list(d.values())

params = list(itertools.product(*params))

print(len(params))

24300


In [70]:
random.seed(1)
params = random.sample(params, 10)

#### Inner loop - training hyperparameter on validation

##### On random space

In [71]:
results_mp = tuning_window_mp(X_fit = X_train, y_fit = y_train, X_test = X_val, y_test = y_val, params = params, n_components = n_components, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [02:56<00:00,  1.71s/it]


In [72]:
with open('results/final/xgboost/results_shap.pickle', 'wb') as handle:
    pickle.dump(results_mp, handle, protocol= pickle.HIGHEST_PROTOCOL)

#### Outer loop - fitting on train / test split

##### On full sample

Reloading data and concatting:

In [73]:
with open('results/final/xgboost/results_shap.pickle', 'rb') as handle:
    results_opt = pickle.load(handle)
    

In [74]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [75]:
#Concatting val and train
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

Standardizing features for each window

In [76]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_test[win] = sc.transform(X_test[win])

In [77]:
results_final = final_model(inner_results=results_opt, X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [00:18<00:00,  5.48it/s]


#### Exporting final results

In [78]:
with open('results/final/xgboost/results_final_shap.pickle', 'wb') as handle:
    pickle.dump(results_final, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [79]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

0.13276080889729835

# Removing data tests

## No GT

### Import data frame with adjusted here

In [2]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])

### Initial preprocessing and feature construction

- Create dummies 
- Create interaction terms

Overall monthly time trend variable, $t=1,2...,T$ within `ID` variable:

In [3]:
#Temp container
temp = {}

for i in df_analysis['ID'].unique():
    temp[i] = df_analysis[df_analysis['ID']==i]
    temp[i]['t'] = range(1, len(temp[i]['ID'])+1)

#Concatting the df's
temp = pd.concat(temp, ignore_index=True, sort = False)

#Merging onto analysis
df_analysis = pd.merge(left = df_analysis, right = temp[['date', 'ID', 't']], left_on =['date', 'ID'], right_on = ['date', 'ID'])

C:\Users\WSCHUPPLI\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Drop sector variables

In [4]:
df_analysis.drop(['sector_sales_communication', 'sector_other'], axis = 1, inplace=True)

#### Dropping socioeconomic index

In [5]:
df_analysis.drop(['w_ave_socio_index', 'target_lag'], axis = 1, inplace = True)

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [6]:
df_analysis['target_actual'] = df_analysis.groupby(['ID'])['target_actual'].diff()

### Lagged variables

In [7]:
df_analysis['target_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(1)

In [8]:
df_analysis['target_12_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(12)

#### Create new variables with 3 month lag of jobrate

In [9]:
columns_3m_lag = ['jobs', 'sector_information_technology', 'sector_engineering_technology',
                   'sector_management_staff', 'sector_trade_service', 'sector_industry_craft',
                   'sector_teaching', 'sector_office_finance', 'sector_social_health']

for colname in columns_3m_lag:
    df_analysis[str(colname + '_3_lag')] = df_analysis.groupby(['ID'])[colname].shift(3)

#### Dropping some GT variables

In [10]:
# Dropping some GT's
drop_list = ['GT_DK_1', 'GT_DK_4', 'GT_DK_5', 'GT_DK_6', 'GT_DK_8', 'GT_DK_11', 'GT_DK_13', 'GT_DK_15', 'GT_DK_16', 'GT_DK_18', 'GT_DK_19', 'GT_DK_21']

In [11]:
df_analysis.drop(drop_list, axis = 1, inplace=True)

#### Create new variables with 1 month lag of GT

In [12]:
columns_1m_lag = ['GT_DK_0', 'GT_DK_2', 'GT_DK_3', 'GT_DK_7', 'GT_DK_9', 'GT_DK_10', 'GT_DK_12', 'GT_DK_14', 'GT_DK_17', 'GT_DK_20']
for colname in columns_1m_lag:
    df_analysis[str(colname + '_1_lag')] = df_analysis.groupby(['ID'])[colname].shift(1)

#### Month dummies for season effects

In [13]:
df_analysis['month'] = pd.DatetimeIndex(df_analysis['date']).month.astype(str)

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [14]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID','M'], prefix_sep='_', columns=['ID', 'month']).copy()

#### Drop na

In [15]:
df_analysis.dropna(inplace=True)

In [16]:
df_analysis.date.min()

Timestamp('2008-03-01 00:00:00')

#### Adding interaction terms

Polynominal features - To be deleted later

In [17]:
#df_analysis = add_poly_terms(df = df_analysis, 
#                            poly_columns = ['target_actual', 'GT_0', 'GT_1', 'GT_2', 'GT_3', 'GT_4', 'GT_5', 'GT_6', 'GT_7', 'GT_8', 'GT_9', 'GT_10', 'GT_11', 'GT_12', 'GT_13', 'GT_14', 'GT_15', 'GT_16', 'GT_17', 'GT_18', 'GT_19', 'target_lag', 'jobs', 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'])

In [18]:
#df_analysis.dropna(inplace=True)

Adding interaction terms by regions and all variables

In [19]:
# relevant interaction variables
interaction_1 = ['target_lag', 'target_12_lag'] 
# 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'

# get list of all ID area 
interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [20]:
for var1 in interaction_1:
    for var2 in interaction_2:
        name = var1 + "*" + var2
        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop variables to not end up in dummytrap

In [21]:
df_analysis = df_analysis.drop(['ID_Capital', 'M_1'], axis = 1)

In [22]:
df_analysis.drop(interaction_1, axis = 1, inplace=True)

In [23]:
df_analysis.sort_index(axis=1, inplace=True)

In [24]:
df_analysis.columns

Index(['GT_DK_0', 'GT_DK_0_1_lag', 'GT_DK_10', 'GT_DK_10_1_lag', 'GT_DK_12',
       'GT_DK_12_1_lag', 'GT_DK_14', 'GT_DK_14_1_lag', 'GT_DK_17',
       'GT_DK_17_1_lag', 'GT_DK_2', 'GT_DK_20', 'GT_DK_20_1_lag',
       'GT_DK_2_1_lag', 'GT_DK_3', 'GT_DK_3_1_lag', 'GT_DK_7', 'GT_DK_7_1_lag',
       'GT_DK_9', 'GT_DK_9_1_lag', 'ID_Central Denmark', 'ID_North Denmark',
       'ID_Southern Denmark', 'ID_Zealand', 'M_10', 'M_11', 'M_12', 'M_2',
       'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'date', 'jobs',
       'jobs_3_lag', 'labour_force_share', 'mvu_lvu_share_pop', 'pop',
       'sector_engineering_technology', 'sector_engineering_technology_3_lag',
       'sector_industry_craft', 'sector_industry_craft_3_lag',
       'sector_information_technology', 'sector_information_technology_3_lag',
       'sector_management_staff', 'sector_management_staff_3_lag',
       'sector_office_finance', 'sector_office_finance_3_lag',
       'sector_social_health', 'sector_social_health_3_lag', 'se

In [25]:
drop_list = ['GT_DK_0', 'GT_DK_0_1_lag', 'GT_DK_10', 'GT_DK_10_1_lag', 'GT_DK_12',
       'GT_DK_12_1_lag', 'GT_DK_14', 'GT_DK_14_1_lag', 'GT_DK_17',
       'GT_DK_17_1_lag', 'GT_DK_2', 'GT_DK_20', 'GT_DK_20_1_lag',
       'GT_DK_2_1_lag', 'GT_DK_3', 'GT_DK_3_1_lag', 'GT_DK_7', 'GT_DK_7_1_lag',
       'GT_DK_9', 'GT_DK_9_1_lag']

In [26]:
df_analysis.drop(drop_list, axis = 1, inplace = True)

#### Setting window size

In [27]:
window = 35
testsize = 1
valsize = 1
rolling_window = True

### XGboost

In [28]:
X_train, X_val, X_test, y_train, y_val, y_test, y_dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

### Pre-processing

- Standardizing

Standardizing features for each window

In [29]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_val[win] = sc.transform(X_val[win])
    X_test[win] = sc.transform(X_test[win])

### Training the models

#### Hyperparameter space - random

In [30]:
# Website https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# parameter specification
n_components = [0.9]


colsample_bytree = [0.3, 0.5, 0.7, 0.9, 1]#is the subsample ratio of columns when constructing each tree. Subsampling will occur once in every boosting iteration. This number ranges from 0 to 1.
#learning_rate is the step size shrinkage and is used to prevent overfitting. This number ranges from 0 to 1.
# Maximum number of levels in tree
# First try: colsample_bytree = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

max_depth = [*range(3, 11, 1), *range(20, 100, 20)]

#max_depth = [*range(10, 300, 10)]
# first try max_depth = [*range(50, 500, 20)]

n_estimators = [*range(50, 500, 10)]#is the number of boosted trees to fit
# first try n_estimators = [*range(50, 500, 20)]

gamma = [0]

subsample = [0.5, 0.75, 1]

min_child_weight = [1, 3, 5]

# Create the random grid
d = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'colsample_bytree': colsample_bytree,
    'gamma' : gamma,
    'subsample' : subsample,
    'min_child_weight' : min_child_weight
    }

params = list(d.values())

params = list(itertools.product(*params))

print(len(params))

24300


In [31]:
random.seed(1)
params = random.sample(params, 10)

#### Inner loop - training hyperparameter on validation

##### On random space

In [32]:
results_mp = tuning_window_mp(X_fit = X_train, y_fit = y_train, X_test = X_val, y_test = y_val, params = params, n_components = n_components, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [02:45<00:00,  1.60s/it]


In [33]:
with open('results/no_data/xgboost/results_noGT.pickle', 'wb') as handle:
    pickle.dump(results_mp, handle, protocol= pickle.HIGHEST_PROTOCOL)

#### Outer loop - fitting on train / test split

##### On full sample

Reloading data and concatting:

In [35]:
with open('results/no_data/xgboost/results_noGT.pickle', 'rb') as handle:
    results_opt = pickle.load(handle)
    

In [36]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [37]:
#Concatting val and train
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

Standardizing features for each window

In [38]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_test[win] = sc.transform(X_test[win])

In [39]:
results_final = final_model(inner_results=results_opt, X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [00:15<00:00,  6.70it/s]


#### Exporting final results

In [40]:
with open('results/no_data/xgboost/results_final_noGT.pickle', 'wb') as handle:
    pickle.dump(results_final, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [41]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

0.13023529679233692

## No JOBS

### Import data frame with adjusted here

In [80]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])

### Initial preprocessing and feature construction

- Create dummies 
- Create interaction terms

Overall monthly time trend variable, $t=1,2...,T$ within `ID` variable:

In [81]:
#Temp container
temp = {}

for i in df_analysis['ID'].unique():
    temp[i] = df_analysis[df_analysis['ID']==i]
    temp[i]['t'] = range(1, len(temp[i]['ID'])+1)

#Concatting the df's
temp = pd.concat(temp, ignore_index=True, sort = False)

#Merging onto analysis
df_analysis = pd.merge(left = df_analysis, right = temp[['date', 'ID', 't']], left_on =['date', 'ID'], right_on = ['date', 'ID'])

C:\Users\WSCHUPPLI\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Drop sector variables

In [82]:
df_analysis.drop(['sector_sales_communication', 'sector_other'], axis = 1, inplace=True)

#### Dropping socioeconomic index

In [83]:
df_analysis.drop(['w_ave_socio_index', 'target_lag'], axis = 1, inplace = True)

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [84]:
df_analysis['target_actual'] = df_analysis.groupby(['ID'])['target_actual'].diff()

### Lagged variables

In [85]:
df_analysis['target_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(1)

In [86]:
df_analysis['target_12_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(12)

#### Create new variables with 3 month lag of jobrate

In [87]:
columns_3m_lag = ['jobs', 'sector_information_technology', 'sector_engineering_technology',
                   'sector_management_staff', 'sector_trade_service', 'sector_industry_craft',
                   'sector_teaching', 'sector_office_finance', 'sector_social_health']

for colname in columns_3m_lag:
    df_analysis[str(colname + '_3_lag')] = df_analysis.groupby(['ID'])[colname].shift(3)

#### Dropping some GT variables

In [88]:
# Dropping some GT's
drop_list = ['GT_DK_1', 'GT_DK_4', 'GT_DK_5', 'GT_DK_6', 'GT_DK_8', 'GT_DK_11', 'GT_DK_13', 'GT_DK_15', 'GT_DK_16', 'GT_DK_18', 'GT_DK_19', 'GT_DK_21']

In [89]:
df_analysis.drop(drop_list, axis = 1, inplace=True)

#### Create new variables with 1 month lag of GT

In [90]:
columns_1m_lag = ['GT_DK_0', 'GT_DK_2', 'GT_DK_3', 'GT_DK_7', 'GT_DK_9', 'GT_DK_10', 'GT_DK_12', 'GT_DK_14', 'GT_DK_17', 'GT_DK_20']
for colname in columns_1m_lag:
    df_analysis[str(colname + '_1_lag')] = df_analysis.groupby(['ID'])[colname].shift(1)

#### Month dummies for season effects

In [91]:
df_analysis['month'] = pd.DatetimeIndex(df_analysis['date']).month.astype(str)

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [92]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID','M'], prefix_sep='_', columns=['ID', 'month']).copy()

#### Drop na

In [93]:
df_analysis.dropna(inplace=True)

In [94]:
df_analysis.date.min()

Timestamp('2008-03-01 00:00:00')

#### Adding interaction terms

Polynominal features - To be deleted later

In [95]:
#df_analysis = add_poly_terms(df = df_analysis, 
#                            poly_columns = ['target_actual', 'GT_0', 'GT_1', 'GT_2', 'GT_3', 'GT_4', 'GT_5', 'GT_6', 'GT_7', 'GT_8', 'GT_9', 'GT_10', 'GT_11', 'GT_12', 'GT_13', 'GT_14', 'GT_15', 'GT_16', 'GT_17', 'GT_18', 'GT_19', 'target_lag', 'jobs', 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'])

In [96]:
#df_analysis.dropna(inplace=True)

Adding interaction terms by regions and all variables

In [97]:
# relevant interaction variables
interaction_1 = ['target_lag', 'target_12_lag'] 
# 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'

# get list of all ID area 
interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [98]:
for var1 in interaction_1:
    for var2 in interaction_2:
        name = var1 + "*" + var2
        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop variables to not end up in dummytrap

In [99]:
df_analysis = df_analysis.drop(['ID_Capital', 'M_1'], axis = 1)

In [100]:
df_analysis.drop(interaction_1, axis = 1, inplace=True)

In [101]:
df_analysis.sort_index(axis=1, inplace=True)

In [105]:
df_analysis.columns

Index(['GT_DK_0', 'GT_DK_0_1_lag', 'GT_DK_10', 'GT_DK_10_1_lag', 'GT_DK_12',
       'GT_DK_12_1_lag', 'GT_DK_14', 'GT_DK_14_1_lag', 'GT_DK_17',
       'GT_DK_17_1_lag', 'GT_DK_2', 'GT_DK_20', 'GT_DK_20_1_lag',
       'GT_DK_2_1_lag', 'GT_DK_3', 'GT_DK_3_1_lag', 'GT_DK_7', 'GT_DK_7_1_lag',
       'GT_DK_9', 'GT_DK_9_1_lag', 'ID_Central Denmark', 'ID_North Denmark',
       'ID_Southern Denmark', 'ID_Zealand', 'M_10', 'M_11', 'M_12', 'M_2',
       'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'date',
       'labour_force_share', 'mvu_lvu_share_pop', 'pop', 't',
       'target_12_lag*ID_Capital', 'target_12_lag*ID_Central Denmark',
       'target_12_lag*ID_North Denmark', 'target_12_lag*ID_Southern Denmark',
       'target_12_lag*ID_Zealand', 'target_actual', 'target_lag*ID_Capital',
       'target_lag*ID_Central Denmark', 'target_lag*ID_North Denmark',
       'target_lag*ID_Southern Denmark', 'target_lag*ID_Zealand',
       'w_ave_urban_index'],
      dtype='object')

In [103]:
drop_list = ['jobs', 'jobs_3_lag', 'sector_engineering_technology', 'sector_engineering_technology_3_lag',
       'sector_industry_craft', 'sector_industry_craft_3_lag',
       'sector_information_technology', 'sector_information_technology_3_lag',
       'sector_management_staff', 'sector_management_staff_3_lag',
       'sector_office_finance', 'sector_office_finance_3_lag',
       'sector_social_health', 'sector_social_health_3_lag', 'sector_teaching',
       'sector_teaching_3_lag', 'sector_trade_service',
       'sector_trade_service_3_lag']

In [104]:
df_analysis.drop(drop_list, axis = 1, inplace = True)

#### Setting window size

In [106]:
window = 35
testsize = 1
valsize = 1
rolling_window = True

### XGboost

In [107]:
X_train, X_val, X_test, y_train, y_val, y_test, y_dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

### Pre-processing

- Standardizing

Standardizing features for each window

In [108]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_val[win] = sc.transform(X_val[win])
    X_test[win] = sc.transform(X_test[win])

### Training the models

#### Hyperparameter space - random

In [109]:
# Website https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# parameter specification
n_components = [0.9]


colsample_bytree = [0.3, 0.5, 0.7, 0.9, 1]#is the subsample ratio of columns when constructing each tree. Subsampling will occur once in every boosting iteration. This number ranges from 0 to 1.
#learning_rate is the step size shrinkage and is used to prevent overfitting. This number ranges from 0 to 1.
# Maximum number of levels in tree
# First try: colsample_bytree = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

max_depth = [*range(3, 11, 1), *range(20, 100, 20)]

#max_depth = [*range(10, 300, 10)]
# first try max_depth = [*range(50, 500, 20)]

n_estimators = [*range(50, 500, 10)]#is the number of boosted trees to fit
# first try n_estimators = [*range(50, 500, 20)]

gamma = [0]

subsample = [0.5, 0.75, 1]

min_child_weight = [1, 3, 5]

# Create the random grid
d = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'colsample_bytree': colsample_bytree,
    'gamma' : gamma,
    'subsample' : subsample,
    'min_child_weight' : min_child_weight
    }

params = list(d.values())

params = list(itertools.product(*params))

print(len(params))

24300


In [110]:
random.seed(1)
params = random.sample(params, 10)

#### Inner loop - training hyperparameter on validation

##### On random space

In [111]:
results_mp = tuning_window_mp(X_fit = X_train, y_fit = y_train, X_test = X_val, y_test = y_val, params = params, n_components = n_components, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [02:41<00:00,  1.57s/it]


In [112]:
with open('results/no_data/xgboost/results_noJOB.pickle', 'wb') as handle:
    pickle.dump(results_mp, handle, protocol= pickle.HIGHEST_PROTOCOL)

#### Outer loop - fitting on train / test split

##### On full sample

Reloading data and concatting:

In [113]:
with open('results/no_data/xgboost/results_noJOB.pickle', 'rb') as handle:
    results_opt = pickle.load(handle)
    

In [114]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [115]:
#Concatting val and train
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

Standardizing features for each window

In [116]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_test[win] = sc.transform(X_test[win])

In [117]:
results_final = final_model(inner_results=results_opt, X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [00:16<00:00,  6.14it/s]


#### Exporting final results

In [118]:
with open('results/no_data/xgboost/results_final_noJOB.pickle', 'wb') as handle:
    pickle.dump(results_final, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [119]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

0.13738561634886023

## No GT + JOBS

### Import data frame with adjusted here

In [120]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])

### Initial preprocessing and feature construction

- Create dummies 
- Create interaction terms

Overall monthly time trend variable, $t=1,2...,T$ within `ID` variable:

In [121]:
#Temp container
temp = {}

for i in df_analysis['ID'].unique():
    temp[i] = df_analysis[df_analysis['ID']==i]
    temp[i]['t'] = range(1, len(temp[i]['ID'])+1)

#Concatting the df's
temp = pd.concat(temp, ignore_index=True, sort = False)

#Merging onto analysis
df_analysis = pd.merge(left = df_analysis, right = temp[['date', 'ID', 't']], left_on =['date', 'ID'], right_on = ['date', 'ID'])

C:\Users\WSCHUPPLI\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Drop sector variables

In [122]:
df_analysis.drop(['sector_sales_communication', 'sector_other'], axis = 1, inplace=True)

#### Dropping socioeconomic index

In [123]:
df_analysis.drop(['w_ave_socio_index', 'target_lag'], axis = 1, inplace = True)

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [124]:
df_analysis['target_actual'] = df_analysis.groupby(['ID'])['target_actual'].diff()

### Lagged variables

In [125]:
df_analysis['target_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(1)

In [126]:
df_analysis['target_12_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(12)

#### Create new variables with 3 month lag of jobrate

In [127]:
columns_3m_lag = ['jobs', 'sector_information_technology', 'sector_engineering_technology',
                   'sector_management_staff', 'sector_trade_service', 'sector_industry_craft',
                   'sector_teaching', 'sector_office_finance', 'sector_social_health']

for colname in columns_3m_lag:
    df_analysis[str(colname + '_3_lag')] = df_analysis.groupby(['ID'])[colname].shift(3)

#### Dropping some GT variables

In [128]:
# Dropping some GT's
drop_list = ['GT_DK_1', 'GT_DK_4', 'GT_DK_5', 'GT_DK_6', 'GT_DK_8', 'GT_DK_11', 'GT_DK_13', 'GT_DK_15', 'GT_DK_16', 'GT_DK_18', 'GT_DK_19', 'GT_DK_21']

In [129]:
df_analysis.drop(drop_list, axis = 1, inplace=True)

#### Create new variables with 1 month lag of GT

In [130]:
columns_1m_lag = ['GT_DK_0', 'GT_DK_2', 'GT_DK_3', 'GT_DK_7', 'GT_DK_9', 'GT_DK_10', 'GT_DK_12', 'GT_DK_14', 'GT_DK_17', 'GT_DK_20']
for colname in columns_1m_lag:
    df_analysis[str(colname + '_1_lag')] = df_analysis.groupby(['ID'])[colname].shift(1)

#### Month dummies for season effects

In [131]:
df_analysis['month'] = pd.DatetimeIndex(df_analysis['date']).month.astype(str)

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [132]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID','M'], prefix_sep='_', columns=['ID', 'month']).copy()

#### Drop na

In [133]:
df_analysis.dropna(inplace=True)

In [134]:
df_analysis.date.min()

Timestamp('2008-03-01 00:00:00')

#### Adding interaction terms

Polynominal features - To be deleted later

In [135]:
#df_analysis = add_poly_terms(df = df_analysis, 
#                            poly_columns = ['target_actual', 'GT_0', 'GT_1', 'GT_2', 'GT_3', 'GT_4', 'GT_5', 'GT_6', 'GT_7', 'GT_8', 'GT_9', 'GT_10', 'GT_11', 'GT_12', 'GT_13', 'GT_14', 'GT_15', 'GT_16', 'GT_17', 'GT_18', 'GT_19', 'target_lag', 'jobs', 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'])

In [136]:
#df_analysis.dropna(inplace=True)

Adding interaction terms by regions and all variables

In [137]:
# relevant interaction variables
interaction_1 = ['target_lag', 'target_12_lag'] 
# 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'

# get list of all ID area 
interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [138]:
for var1 in interaction_1:
    for var2 in interaction_2:
        name = var1 + "*" + var2
        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop variables to not end up in dummytrap

In [139]:
df_analysis = df_analysis.drop(['ID_Capital', 'M_1'], axis = 1)

In [140]:
df_analysis.drop(interaction_1, axis = 1, inplace=True)

In [141]:
df_analysis.sort_index(axis=1, inplace=True)

In [145]:
df_analysis.columns

Index(['ID_Central Denmark', 'ID_North Denmark', 'ID_Southern Denmark',
       'ID_Zealand', 'M_10', 'M_11', 'M_12', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6',
       'M_7', 'M_8', 'M_9', 'date', 'labour_force_share', 'mvu_lvu_share_pop',
       'pop', 't', 'target_12_lag*ID_Capital',
       'target_12_lag*ID_Central Denmark', 'target_12_lag*ID_North Denmark',
       'target_12_lag*ID_Southern Denmark', 'target_12_lag*ID_Zealand',
       'target_actual', 'target_lag*ID_Capital',
       'target_lag*ID_Central Denmark', 'target_lag*ID_North Denmark',
       'target_lag*ID_Southern Denmark', 'target_lag*ID_Zealand',
       'w_ave_urban_index'],
      dtype='object')

In [143]:
drop_list = ['jobs', 'jobs_3_lag', 'sector_engineering_technology', 'sector_engineering_technology_3_lag',
       'sector_industry_craft', 'sector_industry_craft_3_lag',
       'sector_information_technology', 'sector_information_technology_3_lag',
       'sector_management_staff', 'sector_management_staff_3_lag',
       'sector_office_finance', 'sector_office_finance_3_lag',
       'sector_social_health', 'sector_social_health_3_lag', 'sector_teaching',
       'sector_teaching_3_lag', 'sector_trade_service',
       'sector_trade_service_3_lag', 'GT_DK_0', 'GT_DK_0_1_lag', 'GT_DK_10', 'GT_DK_10_1_lag', 'GT_DK_12',
       'GT_DK_12_1_lag', 'GT_DK_14', 'GT_DK_14_1_lag', 'GT_DK_17',
       'GT_DK_17_1_lag', 'GT_DK_2', 'GT_DK_20', 'GT_DK_20_1_lag',
       'GT_DK_2_1_lag', 'GT_DK_3', 'GT_DK_3_1_lag', 'GT_DK_7', 'GT_DK_7_1_lag',
       'GT_DK_9', 'GT_DK_9_1_lag']

In [144]:
df_analysis.drop(drop_list, axis = 1, inplace = True)

#### Setting window size

In [146]:
window = 35
testsize = 1
valsize = 1
rolling_window = True

### XGboost

In [147]:
X_train, X_val, X_test, y_train, y_val, y_test, y_dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

### Pre-processing

- Standardizing

Standardizing features for each window

In [148]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_val[win] = sc.transform(X_val[win])
    X_test[win] = sc.transform(X_test[win])

### Training the models

#### Hyperparameter space - random

In [149]:
# Website https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# parameter specification
n_components = [0.9]


colsample_bytree = [0.3, 0.5, 0.7, 0.9, 1]#is the subsample ratio of columns when constructing each tree. Subsampling will occur once in every boosting iteration. This number ranges from 0 to 1.
#learning_rate is the step size shrinkage and is used to prevent overfitting. This number ranges from 0 to 1.
# Maximum number of levels in tree
# First try: colsample_bytree = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

max_depth = [*range(3, 11, 1), *range(20, 100, 20)]

#max_depth = [*range(10, 300, 10)]
# first try max_depth = [*range(50, 500, 20)]

n_estimators = [*range(50, 500, 10)]#is the number of boosted trees to fit
# first try n_estimators = [*range(50, 500, 20)]

gamma = [0]

subsample = [0.5, 0.75, 1]

min_child_weight = [1, 3, 5]

# Create the random grid
d = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'colsample_bytree': colsample_bytree,
    'gamma' : gamma,
    'subsample' : subsample,
    'min_child_weight' : min_child_weight
    }

params = list(d.values())

params = list(itertools.product(*params))

print(len(params))

24300


In [150]:
random.seed(1)
params = random.sample(params, 10)

#### Inner loop - training hyperparameter on validation

##### On random space

In [151]:
results_mp = tuning_window_mp(X_fit = X_train, y_fit = y_train, X_test = X_val, y_test = y_val, params = params, n_components = n_components, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [02:21<00:00,  1.37s/it]


In [152]:
with open('results/no_data/xgboost/results_noGTJOB.pickle', 'wb') as handle:
    pickle.dump(results_mp, handle, protocol= pickle.HIGHEST_PROTOCOL)

#### Outer loop - fitting on train / test split

##### On full sample

Reloading data and concatting:

In [153]:
with open('results/no_data/xgboost/results_noGTJOB.pickle', 'rb') as handle:
    results_opt = pickle.load(handle)
    

In [154]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [155]:
#Concatting val and train
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

Standardizing features for each window

In [156]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_test[win] = sc.transform(X_test[win])

In [157]:
results_final = final_model(inner_results=results_opt, X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [00:14<00:00,  7.08it/s]


#### Exporting final results

In [158]:
with open('results/no_data/xgboost/results_final_noGTJOB.pickle', 'wb') as handle:
    pickle.dump(results_final, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [159]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

0.12650599499407195

## No GT + JOBS + CONTROLS

### Import data frame with adjusted here

In [307]:
#Loading adjusted df_analysis
df_analysis = pd.read_csv('data/df_DK.csv', sep = ',', parse_dates = ['date'])

### Initial preprocessing and feature construction

- Create dummies 
- Create interaction terms

Overall monthly time trend variable, $t=1,2...,T$ within `ID` variable:

In [308]:
#Temp container
temp = {}

for i in df_analysis['ID'].unique():
    temp[i] = df_analysis[df_analysis['ID']==i]
    temp[i]['t'] = range(1, len(temp[i]['ID'])+1)

#Concatting the df's
temp = pd.concat(temp, ignore_index=True, sort = False)

#Merging onto analysis
df_analysis = pd.merge(left = df_analysis, right = temp[['date', 'ID', 't']], left_on =['date', 'ID'], right_on = ['date', 'ID'])

C:\Users\WSCHUPPLI\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Drop sector variables

In [309]:
df_analysis.drop(['sector_sales_communication', 'sector_other'], axis = 1, inplace=True)

#### Dropping socioeconomic index

In [310]:
df_analysis.drop(['w_ave_socio_index', 'target_lag'], axis = 1, inplace = True)

#### Transform relevant columns to abs change exept those with M_ and ID_, date and t

In [311]:
df_analysis['target_actual'] = df_analysis.groupby(['ID'])['target_actual'].diff()

### Lagged variables

In [312]:
df_analysis['target_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(1)

In [313]:
df_analysis['target_12_lag'] = df_analysis.groupby(['ID'])['target_actual'].shift(12)

#### Create new variables with 3 month lag of jobrate

In [314]:
columns_3m_lag = ['jobs', 'sector_information_technology', 'sector_engineering_technology',
                   'sector_management_staff', 'sector_trade_service', 'sector_industry_craft',
                   'sector_teaching', 'sector_office_finance', 'sector_social_health']

for colname in columns_3m_lag:
    df_analysis[str(colname + '_3_lag')] = df_analysis.groupby(['ID'])[colname].shift(3)

#### Dropping some GT variables

In [315]:
# Dropping some GT's
drop_list = ['GT_DK_1', 'GT_DK_4', 'GT_DK_5', 'GT_DK_6', 'GT_DK_8', 'GT_DK_11', 'GT_DK_13', 'GT_DK_15', 'GT_DK_16', 'GT_DK_18', 'GT_DK_19', 'GT_DK_21']

In [316]:
df_analysis.drop(drop_list, axis = 1, inplace=True)

#### Create new variables with 1 month lag of GT

In [317]:
columns_1m_lag = ['GT_DK_0', 'GT_DK_2', 'GT_DK_3', 'GT_DK_7', 'GT_DK_9', 'GT_DK_10', 'GT_DK_12', 'GT_DK_14', 'GT_DK_17', 'GT_DK_20']
for colname in columns_1m_lag:
    df_analysis[str(colname + '_1_lag')] = df_analysis.groupby(['ID'])[colname].shift(1)

#### Month dummies for season effects

In [318]:
df_analysis['month'] = pd.DatetimeIndex(df_analysis['date']).month.astype(str)

#### Creating dummies from categorial variables - remember to drop the reference category (done after change is constructed)

In [319]:
df_analysis = pd.get_dummies(df_analysis, prefix=['ID','M'], prefix_sep='_', columns=['ID', 'month']).copy()

#### Drop na

In [320]:
df_analysis.dropna(inplace=True)

In [321]:
df_analysis.date.min()

Timestamp('2008-03-01 00:00:00')

#### Adding interaction terms

Polynominal features - To be deleted later

In [322]:
#df_analysis = add_poly_terms(df = df_analysis, 
#                            poly_columns = ['target_actual', 'GT_0', 'GT_1', 'GT_2', 'GT_3', 'GT_4', 'GT_5', 'GT_6', 'GT_7', 'GT_8', 'GT_9', 'GT_10', 'GT_11', 'GT_12', 'GT_13', 'GT_14', 'GT_15', 'GT_16', 'GT_17', 'GT_18', 'GT_19', 'target_lag', 'jobs', 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'])

In [323]:
#df_analysis.dropna(inplace=True)

Adding interaction terms by regions and all variables

In [324]:
# relevant interaction variables
interaction_1 = ['target_lag', 'target_12_lag'] 
# 'sector_information_technology', 'sector_engineering_technology', 'sector_management_staff', 'sector_trade_service', 'sector_industry_craft', 'sector_sales_communication', 'sector_teaching', 'sector_office_finance', 'sector_social_health', 'sector_other'

# get list of all ID area 
interaction_2 = [item for item in df_analysis if item.startswith('ID_')]

In [325]:
for var1 in interaction_1:
    for var2 in interaction_2:
        name = var1 + "*" + var2
        df_analysis[name] = pd.Series(df_analysis[var1] * df_analysis[var2], name=name)

#### Drop variables to not end up in dummytrap

In [326]:
df_analysis = df_analysis.drop(['ID_Capital', 'M_1'], axis = 1)

In [327]:
df_analysis.drop(interaction_1, axis = 1, inplace=True)

In [328]:
df_analysis.sort_index(axis=1, inplace=True)

In [329]:
df_analysis.columns

Index(['GT_DK_0', 'GT_DK_0_1_lag', 'GT_DK_10', 'GT_DK_10_1_lag', 'GT_DK_12',
       'GT_DK_12_1_lag', 'GT_DK_14', 'GT_DK_14_1_lag', 'GT_DK_17',
       'GT_DK_17_1_lag', 'GT_DK_2', 'GT_DK_20', 'GT_DK_20_1_lag',
       'GT_DK_2_1_lag', 'GT_DK_3', 'GT_DK_3_1_lag', 'GT_DK_7', 'GT_DK_7_1_lag',
       'GT_DK_9', 'GT_DK_9_1_lag', 'ID_Central Denmark', 'ID_North Denmark',
       'ID_Southern Denmark', 'ID_Zealand', 'M_10', 'M_11', 'M_12', 'M_2',
       'M_3', 'M_4', 'M_5', 'M_6', 'M_7', 'M_8', 'M_9', 'date', 'jobs',
       'jobs_3_lag', 'labour_force_share', 'mvu_lvu_share_pop', 'pop',
       'sector_engineering_technology', 'sector_engineering_technology_3_lag',
       'sector_industry_craft', 'sector_industry_craft_3_lag',
       'sector_information_technology', 'sector_information_technology_3_lag',
       'sector_management_staff', 'sector_management_staff_3_lag',
       'sector_office_finance', 'sector_office_finance_3_lag',
       'sector_social_health', 'sector_social_health_3_lag', 'se

In [330]:
drop_list = ['jobs', 'jobs_3_lag', 'sector_engineering_technology', 'sector_engineering_technology_3_lag',
       'sector_industry_craft', 'sector_industry_craft_3_lag',
       'sector_information_technology', 'sector_information_technology_3_lag',
       'sector_management_staff', 'sector_management_staff_3_lag',
       'sector_office_finance', 'sector_office_finance_3_lag',
       'sector_social_health', 'sector_social_health_3_lag', 'sector_teaching',
       'sector_teaching_3_lag', 'sector_trade_service',
       'sector_trade_service_3_lag', 'GT_DK_0', 'GT_DK_0_1_lag', 'GT_DK_10', 'GT_DK_10_1_lag', 'GT_DK_12',
       'GT_DK_12_1_lag', 'GT_DK_14', 'GT_DK_14_1_lag', 'GT_DK_17',
       'GT_DK_17_1_lag', 'GT_DK_2', 'GT_DK_20', 'GT_DK_20_1_lag',
       'GT_DK_2_1_lag', 'GT_DK_3', 'GT_DK_3_1_lag', 'GT_DK_7', 'GT_DK_7_1_lag',
       'GT_DK_9', 'GT_DK_9_1_lag', 'w_ave_urban_index', 'labour_force_share', 'mvu_lvu_share_pop', 'pop',
            'M_10', 'M_11', 'M_12', 'M_2', 'M_3', 'M_4', 'M_5', 'M_6',
       'M_7', 'M_8', 'M_9', 't', 'ID_Central Denmark', 'ID_North Denmark', 'ID_Southern Denmark',
       'ID_Zealand']
# 'target_12_lag*ID_Central Denmark', 'target_12_lag*ID_North Denmark', 'target_12_lag*ID_Southern Denmark', 'target_12_lag*ID_Zealand',    'target_12_lag*ID_Capital'

In [331]:
df_analysis.drop(drop_list, axis = 1, inplace = True)

#### Setting window size

In [332]:
window = 35
testsize = 1
valsize = 1
rolling_window = True

### XGboost

In [333]:
X_train, X_val, X_test, y_train, y_val, y_test, y_dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

### Pre-processing

- Standardizing

Standardizing features for each window

In [334]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_val[win] = sc.transform(X_val[win])
    X_test[win] = sc.transform(X_test[win])

### Training the models

#### Hyperparameter space - random

In [335]:
# Website https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# parameter specification
n_components = [0.9]


colsample_bytree = [0.3, 0.5, 0.7, 0.9, 1]#is the subsample ratio of columns when constructing each tree. Subsampling will occur once in every boosting iteration. This number ranges from 0 to 1.
#learning_rate is the step size shrinkage and is used to prevent overfitting. This number ranges from 0 to 1.
# Maximum number of levels in tree
# First try: colsample_bytree = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

max_depth = [*range(3, 11, 1), *range(20, 100, 20)]

#max_depth = [*range(10, 300, 10)]
# first try max_depth = [*range(50, 500, 20)]

n_estimators = [*range(50, 500, 10)]#is the number of boosted trees to fit
# first try n_estimators = [*range(50, 500, 20)]

gamma = [0]

subsample = [0.5, 0.75, 1]

min_child_weight = [1, 3, 5]

# Create the random grid
d = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'colsample_bytree': colsample_bytree,
    'gamma' : gamma,
    'subsample' : subsample,
    'min_child_weight' : min_child_weight
    }

params = list(d.values())

params = list(itertools.product(*params))

print(len(params))

24300


In [336]:
random.seed(1)
params = random.sample(params, 10)

#### Inner loop - training hyperparameter on validation

##### On random space

In [337]:
results_mp = tuning_window_mp(X_fit = X_train, y_fit = y_train, X_test = X_val, y_test = y_val, params = params, n_components = n_components, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [02:03<00:00,  1.20s/it]


In [338]:
with open('results/no_data/xgboost/results_ONLYLAG.pickle', 'wb') as handle:
    pickle.dump(results_mp, handle, protocol= pickle.HIGHEST_PROTOCOL)

#### Outer loop - fitting on train / test split

##### On full sample

Reloading data and concatting:

In [339]:
with open('results/no_data/xgboost/results_ONLYLAG.pickle', 'rb') as handle:
    results_opt = pickle.load(handle)
    

In [340]:
X_train, X_val, X_test, y_train, y_val, y_test, dates = test_train_split(df = df_analysis, window = window, testsize=testsize, valsize = valsize,
                                                                  y_col='target_actual', rolling_window = rolling_window, df_output= False)

In [341]:
#Concatting val and train
for win in X_train.keys():
    X_train[win] = np.concatenate((X_train[win], X_val[win])).copy()
    y_train[win] = np.concatenate((y_train[win], y_val[win])).copy()

Standardizing features for each window

In [342]:
for win in X_train.keys():
    sc = StandardScaler()
    X_train[win] = sc.fit_transform(X_train[win])
    X_test[win] = sc.transform(X_test[win])

In [343]:
results_final = final_model(inner_results=results_opt, X_fit = X_train, y_fit = y_train, X_test = X_test, y_test = y_test, model_str = 'xgboost')

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [00:10<00:00, 10.25it/s]


#### Exporting final results

In [344]:
with open('results/no_data/xgboost/results_final_ONLYLAG.pickle', 'wb') as handle:
    pickle.dump(results_final, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [345]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

0.13723135096243036

In [279]:
temp = []
for key in results_final.keys():
    temp.append(results_final[key]['best_rmse'][1])
np.mean(temp)

0.21281800480529658